
# Data Architecture, Integration and Ingestion

## Trabalho Integrado

### Integrantes do Grupo:
- PAULO KELLER RM359200
- IVAN CARLOS GAZIN - RM359250


### Plataforma

Este notebook foi desenvolvido utilizando a plataforma Databricks, com liguagem spark.



### Objetivo

Realizar análise do arquivo results.csv e responder as perguntas elencadas abaixo

##### 1 - Quantos registros existem na base?
##### 2 - Quantas equipes únicas mandantes existem na base?
##### 3 - Quantas vezes as equipes mandantes saíram vitoriosas?
##### 4 - Quantas vezes as equipes visitantes saíram vitoriosas?
##### 5 - Quantas partidas resultaram em empate?
##### 6 - Quantas partidas foram realizadas em cada país?
##### 7 - Qual país teve mais partidas?
##### 8 - Qual a partida com maior número de gols?
##### 9 - Qual a maior goleada?
##### 10 - Quantos jogos ocorreram no Brasil?

## 1.Dicionário de dados

##### date - data da partida
##### home_team - time mandante
##### away_team - time visitante
##### home_score - placar do time mandante
##### away_score - placar do time visitante
##### tournament - torneio
##### city - cidade onde a partida foi realizada
##### country - país onde a partida foi realizada
##### neutral - desconsiderar

## 2.Carga de dados

In [2]:
from pyspark.sql.functions import *
from pyspark.shell import spark

24/12/16 20:22:01 WARN Utils: Your hostname, Paulos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.15.189 instead (on interface en0)
24/12/16 20:22:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 20:22:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.3
      /_/

Using Python version 3.12.4 (main, Jun 18 2024 10:07:17)
Spark context Web UI available at http://192.168.15.189:4040
Spark context available as 'sc' (master = local[*], app id = local-1734391322366).
SparkSession available as 'spark'.


In [9]:
# File location and type
file_location = "results.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

DataFrame[date: string, home_teamName: string, away_teamName: string, home_scoreHome: string, away_scoreAway: string, tournamentName: string, cityCity: string, countryCountry: string, neutralTRUE: string]

In [10]:
df.cache().count()

40839

In [11]:
df.createOrReplaceTempView('BASE_INICIAL')

In [15]:
# Criação de função contendo o tratamento dos dados, com a formatação dos valores numéricos para INT e criação de 2 novas colunas contento o total de gols da partida e a diferença absoluta de gols entre as equipes

def gera_tabela_tratada():
  df = spark.sql("""
    SELECT
      CAST(date AS DATE)                                  AS DATA_JOGO,
      home_teamName                                       AS EQUIPE_MANDANTE,
      away_teamName                                       AS EQUIPE_VISITANTE,
      CAST(home_scoreHome AS INT)                         AS GOLS_MANDANTE,
      CAST(away_scoreAway AS INT)                         AS GOLS_VISITANTE,
      CAST(home_scoreHome + away_scoreAway AS INT)        AS GOLS_TOTAL,
      CAST(ABS(home_scoreHome - away_scoreAway) AS INT)   AS GOLS_DIFERENCA_ABS,
      tournamentName                                      AS TORNEIO,
      cityCity                                            AS CIDADE,
      countryCountry                                      AS PAIS    
    FROM BASE_INICIAL
    WHERE
        1=1
  """)
  df.createOrReplaceTempView("RESULTADOS_JOGOS")

In [17]:
# Cria comando para execução da função de tratamento de dados

gera_tabela_tratada()

In [27]:
query_result = spark.sql("SELECT * FROM RESULTADOS_JOGOS")
query_result.show()

+----------+----------------+----------------+-------------+--------------+----------+------------------+--------+---------+-------------------+
| DATA_JOGO| EQUIPE_MANDANTE|EQUIPE_VISITANTE|GOLS_MANDANTE|GOLS_VISITANTE|GOLS_TOTAL|GOLS_DIFERENCA_ABS| TORNEIO|   CIDADE|               PAIS|
+----------+----------------+----------------+-------------+--------------+----------+------------------+--------+---------+-------------------+
|1872-11-30|        Scotland|         England|            0|             0|         0|                 0|Friendly|  Glasgow|           Scotland|
|1873-03-08|         England|        Scotland|            4|             2|         6|                 2|Friendly|   London|            England|
|1874-03-07|        Scotland|         England|            2|             1|         3|                 1|Friendly|  Glasgow|           Scotland|
|1875-03-06|         England|        Scotland|            2|             2|         4|                 0|Friendly|   London|      

## 3.Questões e Respostas

-- 1 - Quantos registros existem na base?
-- R: A base possui 40839 registros

In [46]:
query_result = spark.sql("SELECT COUNT(*) AS NR_REGISTROS FROM RESULTADOS_JOGOS")
query_result.show()

+------------+
|NR_REGISTROS|
+------------+
|       40839|
+------------+



-- 2 - Quantas equipes únicas mandantes existem na base?
-- R: A base possui 309 equipes únicas mandantes

In [48]:
query_result = spark.sql("SELECT COUNT(DISTINCT EQUIPE_MANDANTE) AS NR_EQUIPES_MANDANTES FROM RESULTADOS_JOGOS")
query_result.show()

+--------------------+
|NR_EQUIPES_MANDANTES|
+--------------------+
|                 309|
+--------------------+



-- 3 - Quantas vezes as equipes mandantes saíram vitoriosas?
-- R: As equipes mandantes venceram 19856 vezes

In [51]:
query_result = spark.sql("SELECT COUNT(*) AS NR_JOGOS FROM RESULTADOS_JOGOS WHERE GOLS_MANDANTE > GOLS_VISITANTE")
query_result.show()

+--------+
|NR_JOGOS|
+--------+
|   19864|
+--------+



-- 4 - Quantas vezes as equipes visitantes saíram vitoriosas?
-- R: As equipes visitantes venceram 11552 vezes

In [74]:
query_result = spark.sql("SELECT COUNT(*) AS NR_JOGOS FROM RESULTADOS_JOGOS WHERE GOLS_MANDANTE < GOLS_VISITANTE")
query_result.show()

+--------+
|NR_JOGOS|
+--------+
|   11544|
+--------+



-- 5 - Quantas partidas resultaram em empate?
-- R: As partidas resultaram em empates 9431 vezes

In [55]:
query_result = spark.sql("SELECT COUNT(*) AS NR_JOGOS FROM RESULTADOS_JOGOS WHERE GOLS_MANDANTE = GOLS_VISITANTE")
query_result.show()

+--------+
|NR_JOGOS|
+--------+
|    9431|
+--------+



-- 6 - Quantas partidas foram realizadas em cada país?
-- R: Segue resposta na query abaixo (total de 266 países sediaram jogos)

In [76]:
query_result = spark.sql("SELECT PAIS, COUNT(*) AS NR_JOGOS FROM RESULTADOS_JOGOS GROUP BY PAIS")
query_result.show()
query_result.count()

+-------------------+--------+
|               PAIS|NR_JOGOS|
+-------------------+--------+
|               Chad|      40|
|             Russia|     209|
|           Paraguay|     218|
|           Anguilla|      12|
|              Yemen|      58|
|U.S. Virgin Islands|      15|
|     Chinese Taipei|      47|
|            Senegal|     202|
|             Sweden|     637|
|             Guyana|      98|
|        Philippines|      83|
|              Burma|      79|
|             Jersey|      68|
|            Eritrea|      16|
| Netherlands Guyana|      29|
|           Djibouti|      21|
|              Tonga|      12|
|          Singapore|     368|
|           Malaysia|     644|
|               Fiji|     139|
+-------------------+--------+
only showing top 20 rows



266

-- 7 - Qual país teve mais partidas?
-- R: O pais com mais jogos foi United States com 1144 jogos

In [79]:
query_result = spark.sql("""SELECT PAIS, 
NR_JOGOS FROM ( SELECT PAIS, COUNT(*) AS NR_JOGOS FROM RESULTADOS_JOGOS GROUP BY PAIS) 
ORDER BY NR_JOGOS DESC LIMIT 1""")
query_result.show()

+-------------+--------+
|         PAIS|NR_JOGOS|
+-------------+--------+
|United States|    1144|
+-------------+--------+



-- 8 - Qual a partida com maior número de gols?
-- R: A partida com maior número de gols foi realizada no dia 11/04/2021 na Australia entre as equipes Australia e American Samoa pelo torneio FIFA World Cup qualification, com um total de 31 gols

In [84]:
query_result = spark.sql("""SELECT DATA_JOGO, 
EQUIPE_MANDANTE, 
EQUIPE_VISITANTE, 
GOLS_MANDANTE, 
GOLS_VISITANTE, 
GOLS_TOTAL, 
TORNEIO, 
CIDADE, PAIS 
FROM RESULTADOS_JOGOS
ORDER BY
  GOLS_TOTAL DESC
LIMIT 5""")
query_result.show()

+----------+---------------+----------------+-------------+--------------+----------+--------------------+-------------+----------------+
| DATA_JOGO|EQUIPE_MANDANTE|EQUIPE_VISITANTE|GOLS_MANDANTE|GOLS_VISITANTE|GOLS_TOTAL|             TORNEIO|       CIDADE|            PAIS|
+----------+---------------+----------------+-------------+--------------+----------+--------------------+-------------+----------------+
|2001-04-11|      Australia|  American Samoa|           31|             0|        31|FIFA World Cup qu...|Coffs Harbour|       Australia|
|1971-09-13|         Tahiti|    Cook Islands|           30|             0|        30| South Pacific Games|      Papeete|French Polynesia|
|1979-08-30|           Fiji|        Kiribati|           24|             0|        24| South Pacific Games|      Nausori|            Fiji|
|2001-04-09|      Australia|           Tonga|           22|             0|        22|FIFA World Cup qu...|Coffs Harbour|       Australia|
|2006-11-24|          Sápmi|      

-- 9 - Qual a maior goleada?
-- R: A maior goleada foi realizada pela equipe Australia contra American Samoa, com uma diferença de 31 gols

In [87]:
query_result = spark.sql("""SELECT 
  DATA_JOGO,
  EQUIPE_MANDANTE,
  EQUIPE_VISITANTE,
  GOLS_MANDANTE,
  GOLS_VISITANTE,
  GOLS_DIFERENCA_ABS,
  TORNEIO,
  CIDADE,
  PAIS 
FROM RESULTADOS_JOGOS
ORDER BY
  GOLS_DIFERENCA_ABS DESC
LIMIT 5""")
query_result.show()

+----------+---------------+----------------+-------------+--------------+------------------+--------------------+-------------+----------------+
| DATA_JOGO|EQUIPE_MANDANTE|EQUIPE_VISITANTE|GOLS_MANDANTE|GOLS_VISITANTE|GOLS_DIFERENCA_ABS|             TORNEIO|       CIDADE|            PAIS|
+----------+---------------+----------------+-------------+--------------+------------------+--------------------+-------------+----------------+
|2001-04-11|      Australia|  American Samoa|           31|             0|                31|FIFA World Cup qu...|Coffs Harbour|       Australia|
|1971-09-13|         Tahiti|    Cook Islands|           30|             0|                30| South Pacific Games|      Papeete|French Polynesia|
|1979-08-30|           Fiji|        Kiribati|           24|             0|                24| South Pacific Games|      Nausori|            Fiji|
|2001-04-09|      Australia|           Tonga|           22|             0|                22|FIFA World Cup qu...|Coffs Harb

-- 10 - Quantos jogos ocorreram no Brasil?
-- R: No total ocorrem 529 jogos no Brasil

In [92]:
query_result = spark.sql("""SELECT
  PAIS,
  COUNT(*)    AS NR_JOGOS
FROM RESULTADOS_JOGOS
WHERE 
    UPPER(TRIM(PAIS)) = 'BRAZIL'
OR  UPPER(TRIM(PAIS)) = 'BRASIL'
GROUP BY 
  PAIS""")
query_result.show()

+------+--------+
|  PAIS|NR_JOGOS|
+------+--------+
|Brazil|     529|
+------+--------+

